# SciMLOperators.jl

## Simplest Operator: MatrixOperator

`MatrixOperator` just turns a matrix into an `AbstractSciMLOperator`, so it's not really a matrix-free operator but it's a starting point that is good for understanding the interface and testing.

In [21]:
using SciMLOperators, LinearAlgebra

A = [-2.0  1  0  0  0
      1 -2  1  0  0
      0  1 -2  1  0
      0  0  1 -2  1
      0  0  0  1 -2]

opA = MatrixOperator(A)

MatrixOperator(5 × 5)

The operators can do operations as defined in the operator interface, for example, matrix multiplication as the core action:

In [22]:
v = [3.0,2.0,1.0,2.0,3.0]

# Different options:
opA*v
# opA(v, nothing, nothing, nothing) 
w = zeros(5)
mul!(w, opA, v)
α = 1.0; β = 1.0
mul!(w, opA, v, α, β)           # α*opA*v + β*w

5-element Vector{Float64}:
 -8.0
  0.0
  4.0
  0.0
 -8.0

and the inverse operation:

In [23]:
opA \ v
ldiv!(w, lu(opA), v)

5-element Vector{Float64}:
 -5.5
 -7.999999999999999
 -8.499999999999998
 -8.0
 -5.5

## State, Parameter, and Time-Dependent Operators

Now let's define a `MatrixOperator` that is dependent on state, parameters, and time. For example, let's make the operator `A .* u + dt*I` where `dt` is a parameter and `u` is a state vector:

In [24]:
A = [-2.0  1  0  0  0
      1 -2  1  0  0
      0  1 -2  1  0
      0  0  1 -2  1
      0  0  0  1 -2]

function update_function!(B, u, p, t)
    dt = p
    B .= A .* u + dt*I
end

u = Array(1:1.0:5); p = 0.1; t = 0.0
opB = MatrixOperator(copy(A); update_func! = update_function!)

MatrixOperator(5 × 5)

To update the operator, you would use `update_coefficients!(opB, u, p, t)`:

In [25]:
update_coefficients!(opB, u, p, t)

We can use the interface to see what the current matrix is by converting to a standard matrix:

In [26]:
convert(AbstractMatrix, opB)

5×5 Matrix{Float64}:
 -1.9   1.0   0.0   0.0   0.0
  2.0  -3.9   2.0   0.0   0.0
  0.0   3.0  -5.9   3.0   0.0
  0.0   0.0   4.0  -7.9   4.0
  0.0   0.0   0.0   5.0  -9.9

In [27]:
opB*v

5-element Vector{Float64}:
  -3.6999999999999993
   0.20000000000000018
   6.1
   0.1999999999999993
 -19.700000000000003

## Matrix-Free Operators via FunctionOperator

The goal now is to define the function in a matrix-free way:

In [31]:
function Afunc!(w,v,u,p,t)
    w[1] = -2v[1] + v[2]
    for i in 2:4
        w[i] = v[i-1] - 2v[i] + v[i+1]
    end
    w[5] = v[4] - 2v[5]
    nothing
end

function Afunc!(v,u,p,t)
    w = zeros(5)
    Afunc!(w,v,u,p,t)
    w
end

mfopA = FunctionOperator(Afunc!, zeros(5), zeros(5))

FunctionOperator(5 × 5)

In [40]:
mfopA*v

MethodError: MethodError: no method matching *(::FunctionOperator{true, false, false, Float64, typeof(Afunc!), Nothing, Nothing, Nothing, @NamedTuple{islinear::Bool, isconvertible::Bool, isconstant::Bool, opnorm::Nothing, issymmetric::Bool, ishermitian::Bool, isposdef::Bool, isinplace::Bool, outofplace::Bool, has_mul5::Bool, ifcache::Bool, T::DataType, batch::Bool, size::Tuple{Int64, Int64}, sizes::Tuple{Tuple{Int64}, Tuple{Int64}}, accepted_kwargs::Val{()}, kwargs::@NamedTuple{}}, Nothing, Float64, Tuple{Vector{Float64}, Vector{Float64}}, Float64, Float64}, ::Vector{Float64})
The function `*` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:596
  *(!Matched::NullOperator, ::AbstractVecOrMat)
   @ SciMLOperators C:\Users\jonas\.julia\packages\SciMLOperators\2UPBq\src\basic.jl:133
  *(::SciMLOperators.AbstractSciMLOperator, !Matched::IdentityOperator)
   @ SciMLOper